In [38]:
import pandas as pd

In [132]:
def amotized_loan(month,P,i=0.0335,n=360):
    """
    month: month order
    P: amount of principal, net of initial payments
    r: periodic interest rate (Anually) 
    n: total number of payments
    """
    
    r_adj = i/12
    #Periodic payment amount
    A = P*(r_adj*(1+r_adj)**n)/((1+r_adj)**n-1)
    
    #Interest payment by month
    total_paid = 0
    principal_paid = 0
    for i in range(month):
        int_pay_now = (P-total_paid) * r_adj
        princ_pay_now = A - int_pay_now
        
        total_paid = total_paid + princ_pay_now
        print(i+1, int_pay_now, princ_pay_now, total_paid)
    return A, int_pay_now,  princ_pay_now


In [133]:
A, int_pay_now,  princ_pay_now = amotized_loan(360,265000)

1 739.7916666666666 428.09919032685036 428.09919032685036
2 738.5965564270042 429.2943005665128 857.3934908933632
3 737.3981098379227 430.4927471555943 1287.8862380489575
4 736.1963175854466 431.69453940807034 1719.5807774570278
5 734.9911703295991 432.8996866639179 2152.4804641209457
6 733.7826587043292 434.10819828918784 2586.5886624101336
7 732.5707733174384 435.32008367607864 3021.908746086212
8 731.3555047505093 436.53535224300765 3458.4440983292197
9 730.136843558831 437.75401343468604 3896.1981117639057
10 728.9147802713258 438.97607672219124 4335.174188486097
11 727.6893053904763 440.2015516030407 4775.375740089137
12 726.4604093922512 441.43044760126577 5216.806187690403
13 725.228082726031 442.66277426748604 5659.468961957889
14 723.9923158145342 443.8985411789828 6103.367503136872
15 722.753099053743 445.137757939774 6548.505261076646
16 721.5104228128278 446.38043418068924 6994.885695257335
17 720.2642774340733 447.6265795594437 7442.512274816779
18 719.0146532328032 448.87

In [137]:
def int_only_strat(P_0, months, har, ir,
                   mi = 2500, rhr = 0.008, pt = 0.0182, insurance = 1500, month_rent_year = 11.5):
    """
    P_0: Initial House price
    months:  Number of month
    har: House appreciation rate (Anually)
    ir: Interest rate
    
    mi: Monthly investments. Default: 2500
    rhr: rent/house ratio. Default: 0.008
    pt: Annual property tax rate. Default 1.82%
    insurance: Annual insurance. Default 1500
    month_rent_year: number of month rent per year. Default 11.5
    
    ---Return---
    balance
    num_houses
    """
    
    #Initialization
    balance = 0
    num_houses = 1
    summary = {}
    
    #Iteration
    for month in range(1,months+1):
        #Current house price at given month
        P_t = P_0 * (1 + month* har/12) #Non-compound
        
        #Debt
        debt = P_t * 0.8
        
        #Rent profit = monthly rental net - monthly property tax  - monthly insurance- monthly interest
        profit = (month_rent_year/12)*P_t*rhr - P_t*pt/12 - insurance/12 - debt*ir/12 
#         A, int_pay_now,  princ_pay_now = amotized_loan(month, P_0,i=ir,n=360)
#         profit = (month_rent_year/12)*P_t*rhr - A - P_t*pt/12 - insurance/12
        profit = profit * num_houses
        
        #Investment Balance
        balance = balance + profit + mi
        
        #Net worth
        net = num_houses*P_t*0.2+balance
        total_investment = mi*month + P_0 * 0.2
        
        #Summary
        summary.setdefault('month', []).append(month)
        summary.setdefault('house price', []).append(P_t)
        summary.setdefault('profit', []).append(profit)
        summary.setdefault('balance', []).append(balance)
        summary.setdefault('num houses', []).append(num_houses)
        summary.setdefault('invest', []).append(total_investment)
        summary.setdefault('net', []).append(net)
        
        #Buy house when balance is sufficient. (Down 20 percent)
        if balance > P_t*0.2:
            num_houses = num_houses + 1
            balance = balance - P_t*0.2
            
    summary = pd.DataFrame(summary)
    return summary

In [144]:
summary = int_only_strat(P_0=250000, months=84, har=0.05, ir=0.03375, mi = 1000)

In [145]:
summary

,month,house price,profit,balance,num houses,invest,net
0,1,251041.666667,854.0625,1854.0625,1,51000.0,52062.395833
1,2,252083.333333,858.1250,3712.1875,1,52000.0,54128.854167
2,3,253125.000000,862.1875,5574.3750,1,53000.0,56199.375000
3,4,254166.666667,866.2500,7440.6250,1,54000.0,58273.958333
4,5,255208.333333,870.3125,9310.9375,1,55000.0,60352.604167
...,...,...,...,...,...,...,...
79,80,333333.333333,5875.0000,22880.0000,5,130000.0,356213.333333
80,81,334375.000000,5895.3125,29775.3125,5,131000.0,364150.312500
81,82,335416.666667,5915.6250,36690.9375,5,132000.0,372107.604167
82,83,336458.333333,5935.9375,43626.8750,5,133000.0,380085.208333


In [143]:
(680000-260000)/260000

1.6153846153846154